In [1]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show

path = 'results/grid_search_1121_2043/experiment_000738.csv'
df = pd.read_csv(path)


In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
df.columns


Index(['0', 'batch_size', 'lr', 'max_epochs', 'module__activation_fct',
       'module__channel_list', 'module__conv_type', 'module__dropout',
       'module__fc_hidden_layers', 'module__preprocess', 'train_loss_0',
       'train_loss_1', 'train_loss_2', 'train_loss_min', 'train_loss_max',
       'train_loss_mean', 'train_loss_std', 'valid_acc_0', 'valid_acc_1',
       'valid_acc_2', 'valid_acc_min', 'valid_acc_max', 'valid_acc_mean',
       'valid_acc_std', 'valid_loss_0', 'valid_loss_1', 'valid_loss_2',
       'valid_loss_min', 'valid_loss_max', 'valid_loss_mean',
       'valid_loss_std'],
      dtype='object')